In [1]:
# ! pip install pandas
# ! pip install requests
# ! pip install plotly
# ! pip install datetime
# ! pip install os
# ! pip freeze = requirements.txt

In [16]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import os

In [3]:
pwd = os.getcwd()
if 'L2 TVL' in pwd:
    prepend = ''
else:
    prepend = 'L2 TVL/'

In [7]:

api_str = 'https://api.llama.fi/protocol/'

# Protocol Incentive Start Dates
# NOTE: This should be when the in-app incentives began, not any external incentives (i.e. DEX pools)
protocols = [\
    # name
         ['uniswap-v3']
        ,['arrakis-finance']
        ,['gamma']
        ,['xtoken']
        ,['revert-compoundor']
        ]
chains = [
    ['Optimism'],
    ['Arbitrum'],
    ['Ethereum'],
    ['Polygon']
]
# print(protocols[0])
prod = []
for prot in protocols:
    print(prot[0])
    tmp = r.get(api_str + prot[0]).json()
    for ch in chains:
        print(ch[0])
        try:
            tp = tmp['chainTvls'][ch[0] ]
            ad = pd.json_normalize( tp['tokens'] )
            ad_usd = pd.json_normalize( tp['tokensInUsd'] )
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days

                ad['token'] = ad['token'].str.replace('tokens.','', regex=False)
                ad['protocol'] = prot[0]
                ad['chain'] = ch[0]
                # ad['start_date'] = pd.to_datetime(prot[1])
                # ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
        except:
            continue

df_df = pd.concat(prod)

uniswap-v3
Optimism
Arbitrum
Ethereum
Polygon
arrakis-finance
Optimism
Arbitrum
Ethereum
Polygon
gamma
Optimism
Arbitrum
Ethereum
Polygon
xtoken
Optimism
Arbitrum
Ethereum
Polygon
revert-compoundor
Optimism
Arbitrum
Ethereum
Polygon


In [8]:
df_df

,date,token,token_value,usd_value,protocol,chain
0,2021-11-13 00:00:00,UNI,170523.82900,4.206999e+06,uniswap-v3,Optimism
1,2021-11-14 00:00:00,UNI,170330.92749,4.216519e+06,uniswap-v3,Optimism
2,2021-11-15 00:00:00,UNI,170064.19827,4.203667e+06,uniswap-v3,Optimism
3,2021-11-16 00:00:00,UNI,170677.45778,4.111264e+06,uniswap-v3,Optimism
4,2021-11-17 00:00:00,UNI,171514.47540,3.740125e+06,uniswap-v3,Optimism
...,...,...,...,...,...,...
3732,2022-11-07 00:00:00,SAND,1793.46300,1.516165e+03,revert-compoundor,Polygon
3733,2022-11-08 00:00:00,SAND,1793.46300,1.511066e+03,revert-compoundor,Polygon
3734,2022-11-09 00:00:00,SAND,1793.46300,1.272986e+03,revert-compoundor,Polygon
3735,2022-11-10 00:00:00,SAND,1793.46300,1.063981e+03,revert-compoundor,Polygon


In [19]:
data_df = df_df.copy()#merge(cg_df, on=['date','token'],how='inner')
data_df = data_df[data_df['date'] > pd.Timestamp( (date.today()-timedelta(days=90 +1)) ) ]
data_df = data_df[data_df['token_value'] > 0]

data_df.sort_values(by='date',inplace=True)
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

data_df.sort_values(by='date',inplace=True)

data_df['last_token_value'] = data_df.groupby(['token','protocol'])['token_value'].shift(1)
data_df['last_price_usd'] = data_df.groupby(['token','protocol'])['price_usd'].shift(1)
data_df['last_token_value'] = data_df['last_token_value'].fillna(0)

data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']
data_df['net_price_change'] = data_df['price_usd'] - data_df['last_price_usd']

data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']

data_df['net_price_stock_change'] = data_df['last_token_value'] * data_df['net_price_change']


# display(data_df)

In [20]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')
data_df.head()
# data_df[(data_df['protocol'] == 'pooltogether') & (data_df['date'] >= '2022-10-06') & (data_df['date'] <= '2022-10-12')].tail(10)

,date,token,token_value,usd_value,protocol,chain,price_usd,last_token_value,last_price_usd,net_token_flow,net_price_change,net_dollar_flow,net_price_stock_change
16874,2022-08-12,BANK,9.282856e+05,1.641224e+04,arrakis-finance,Ethereum,0.017680,0.0,NaN,9.282856e+05,NaN,1.641224e+04,NaN
741858,2022-08-12,WFLOW,7.082516e+05,2.115409e+06,uniswap-v3,Ethereum,2.986804,0.0,NaN,7.082516e+05,NaN,2.115409e+06,NaN
32,2022-08-12,USDT,3.594387e+02,3.597981e+02,gamma,Optimism,1.001000,0.0,NaN,3.594387e+02,NaN,3.597981e+02,NaN
4783,2022-08-12,HEX,2.523276e+05,1.504481e+04,uniswap-v3,Polygon,0.059624,0.0,NaN,2.523276e+05,NaN,1.504481e+04,NaN
133558,2022-08-12,DUSK,1.179648e+06,1.941542e+05,uniswap-v3,Ethereum,0.164587,0.0,NaN,1.179648e+06,NaN,1.941542e+05,NaN


In [22]:
# netdf_df = data_df[data_df['date']>= data_df['start_date']][['date','protocol','net_dollar_flow','net_price_stock_change','usd_value']]
netdf_df = data_df.copy()

netdf_df = netdf_df.groupby(['date','protocol','chain']).sum(['net_dollar_flow','net_price_stock_change','usd_value'])


netdf_df['tvl_change'] = netdf_df['usd_value'] - netdf_df.groupby(['protocol'])['usd_value'].shift(1)
netdf_df['error'] = netdf_df['tvl_change'] - (netdf_df['net_dollar_flow'] + netdf_df['net_price_stock_change'])

netdf_df['cumul_net_dollar_flow'] = netdf_df['net_dollar_flow'].groupby(['protocol']).cumsum()
netdf_df['cumul_net_price_stock_change'] = netdf_df['net_price_stock_change'].groupby(['protocol']).cumsum()
netdf_df.reset_index(inplace=True)

netdf_df = netdf_df[['date','protocol','chain','usd_value']]

display(netdf_df)

,date,protocol,chain,usd_value
0,2022-08-12 00:00:00,arrakis-finance,Ethereum,1.686559e+09
1,2022-08-12 00:00:00,arrakis-finance,Optimism,4.748576e+06
2,2022-08-12 00:00:00,arrakis-finance,Polygon,3.248846e+06
3,2022-08-12 00:00:00,gamma,Ethereum,4.436528e+06
4,2022-08-12 00:00:00,gamma,Optimism,1.506968e+04
...,...,...,...,...
1556,2022-11-10 22:59:47,uniswap-v3,Optimism,4.460189e+07
1557,2022-11-10 22:59:47,uniswap-v3,Polygon,8.692602e+07
1558,2022-11-10 22:59:47,xtoken,Arbitrum,3.732745e+05
1559,2022-11-10 22:59:47,xtoken,Ethereum,2.030000e+06


In [34]:
comp_df = netdf_df[netdf_df['protocol'] != 'uniswap-v3']
uni_df = netdf_df[netdf_df['protocol'] == 'uniswap-v3']


merge_df = comp_df.merge(uni_df, on = ['date','chain'], how = 'left')
merge_df['share_of_uni_tvl'] = merge_df['usd_value_x'] / merge_df['usd_value_y']
merge_df = merge_df.sort_values(by=['chain','date'])
display(merge_df)


,date,protocol_x,chain,usd_value_x,protocol_y,usd_value_y,share_of_uni_tvl
6,2022-08-12 00:00:00,revert-compoundor,Arbitrum,1.606501e+05,uniswap-v3,7.765434e+07,0.002069
10,2022-08-12 00:00:00,xtoken,Arbitrum,1.639657e+05,uniswap-v3,7.765434e+07,0.002111
19,2022-08-13 00:00:00,revert-compoundor,Arbitrum,1.649137e+05,uniswap-v3,8.016355e+07,0.002057
29,2022-08-14 00:00:00,revert-compoundor,Arbitrum,1.529264e+05,uniswap-v3,7.551680e+07,0.002025
33,2022-08-14 00:00:00,xtoken,Arbitrum,1.666364e+05,uniswap-v3,7.551680e+07,0.002207
...,...,...,...,...,...,...,...
1171,2022-11-10 00:00:00,gamma,Polygon,3.971683e+05,uniswap-v3,8.001818e+07,0.004963
1175,2022-11-10 00:00:00,revert-compoundor,Polygon,1.517708e+05,uniswap-v3,8.001818e+07,0.001897
1181,2022-11-10 22:59:47,arrakis-finance,Polygon,4.070113e+06,uniswap-v3,8.692602e+07,0.046823
1185,2022-11-10 22:59:47,gamma,Polygon,4.326264e+05,uniswap-v3,8.692602e+07,0.004977


In [37]:
chain_color_map={
                "Arbitrum": "#000000",
                "Avalanche": "pink",
                "Solana": "lightgreen",
                "Fantom": "#50ACE0",
                "Optimism": "red",
                "Polygon": "#7648D6",
                "Binance":"#EABC4E",
                "xDai":"darkgreen"
                }

In [46]:
protos = merge_df['protocol_x'].drop_duplicates()
for p in protos:
        plot_df = merge_df[merge_df['protocol_x'] == p]
        fig = px.line(plot_df, x="date", y="share_of_uni_tvl", color="chain", \
                title= p + " : Share of Uniswap V3 TVL by Chain",\
                labels={
                        "date": "Day",
                        "share_of_uni_tvl": "% Share of Uniswap TVL"
                        }
                ,color_discrete_map=chain_color_map
                )
        fig.update_layout(
        legend_title="Chain"
        ,yaxis = dict(tickformat = ',.1%')
        )
        fig.show()

In [ ]:
netdf_df[(netdf_df['protocol'] == 'pooltogether') & (netdf_df['date'] >= '2022-10-06') & (netdf_df['date'] <= '2022-10-12')].tail(10)

In [ ]:
fig = px.line(netdf_df, x="date", y="net_dollar_flow", color="protocol", \
             title="Daily Net Dollar Flow since Program Announcement",\
            labels={
                     "date": "Day",
                     "net_dollar_flow": "Net Dollar Flow (N$F)"
                 }
            )
fig.update_layout(
    legend_title="App Name"
)
fig.update_layout(yaxis_tickprefix = '$')
fig.write_image(prepend + "img_outputs/svg/daily_ndf.svg")
fig.write_image(prepend + "img_outputs/png/daily_ndf.png")
fig.write_html(prepend + "img_outputs/daily_ndf.html", include_plotlyjs='cdn')

# cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
#              title="Cumulative Dollar Flow since Program Announcement",\
#                    labels={
#                      "date": "Day",
#                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
#                  }
#             ,areamode='group')
# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.show()


cumul_fig = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix = '$')
cumul_fig.update_layout(
    title="Cumulative Net Dollar Flow since Program Announcement",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F)",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
cumul_fig.write_image(prepend + "img_outputs/svg/cumul_ndf.svg") #prepend + 
cumul_fig.write_image(prepend + "img_outputs/png/cumul_ndf.png") #prepend + 
cumul_fig.write_html(prepend + "img_outputs/cumul_ndf.html", include_plotlyjs='cdn')
# cumul_fig.show()


In [ ]:
# fig.show()
# cumul_fig.show()
print("yay")

In [50]:
# ! jupyter nbconvert --to python optimism_app_net_flows.ipynb

[NbConvertApp] Converting notebook optimism_app_net_flows.ipynb to python
[NbConvertApp] Writing 6862 bytes to optimism_app_net_flows.py
